# 隐语SecretFlow金融风控全链路能力展示

> This tutorial is only available in Chinese.

> Last updated: May 12, 2023
>
> 请使用v0.8.3或以上版本的隐语进行实验。
>
> 以下代码仅作为示例，请勿在生产环境直接使用。

本次实验将会展示如何使用隐语进行在风控领域常用的Logistic Regeression模型和XGB模型的模型研发工作。

隐语接下来将会开放模型部署和在线/离线模型预测功能，敬请期待。

## 实验目标

在本次实验中，我们将会利用一个开源数据集训练一个金融风控场景常用的线性回归和XGB模型。在此过程中将包含以下步骤：

- 样本对齐
- 特征预处理
- 数据分析
- 模型训练
- 模型预测
- 模型评估

请依次执行所有步骤确保实验可以顺利完成。

## 实验前置工作

### 初始化隐语框架

在本次实验中，我们将会包含两个节点：**alice** 和 **bob** . 在真实业务场景，他们将会代表两个不同实体，他们之间的原始数据不被允许直接相互传输，但是他们的原始数据将会被一起用以研发一个模型。

在下面的代码中，我们建立了一个 **SecretFlow Cluster**, 基于 **alice** 和 **bob** 两个节点，我们还创建了三个device：

- alice: PYU device, 负责在alice侧的本地计算，计算输入、计算过程和计算结果仅alice可见
- bob: PYU device, 负责在bob侧的本地计算，计算输入、计算过程和计算结果仅bob可见
- spu: SPU device, 负责alice和bob之间的密态计算，计算输入和计算结果为密态，由alice和bob各掌握一个分片，计算过程为MPC计算，由alice和bob各自的SPU Runtime一起执行。

>  如果你尚未理解以上的一些概念，比如SPU设备，请参考这篇[文档](../developer/design/architecture.md).


In [1]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

sf.shutdown()
sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
2023-06-19 19:58:52,337	INFO worker.py:1538 -- Started a local Ray instance.


在上面的log中，你应该发现，在**spu**的创建过程中，alice和bob两边都各有一个 **SPURuntime** 被建立并互相创建连接。

### 数据集

本次实验我们采用的原始数据是来自UCI的[Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). 这个数据集汇集了一家葡萄牙银行机构电话营销的结果。

我们添加了**uid**这一列用于接下来隐私求交的实验。

我们首先看一下数据集所包含的信息。


In [2]:
import pandas as pd

# secretflow.utils.simulation.datasets contains mirrors of some popular open dataset.
from secretflow.utils.simulation.datasets import dataset

df = pd.read_csv(dataset('bank_marketing_full'), sep=';')
df['uid'] = df.index + 1

df


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,2
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,3
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,4
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,45207
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,45208
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,45209
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,45210


该数据集包含了45211个样本，每一个样本代表了一个目标客户。

每个样本包含16个feature，我们这里简单描述一下这个数据集所有的feature。


| feature | 描述 | 取值 |
| :-----| :---- | :---- |
| uid | 客户编码 | 数字 |
| age | 年龄 | 数字 |
| job | 工作类型 |  'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown' |
| marital | 婚姻状况 | 'divorced','married','single','unknown' |
| education | 教育状况 | 'tertiary', 'secondary', 'unknown', 'primary' |
| default | 是否有不良信用记录 | 'no','yes','unknown' |
| housing | 是否有房贷 |  'no','yes','unknown' |
| loan | 是否有个人贷款 | 'no','yes','unknown' |
| contact | 联系方式 | 'cellular','telephone' |
| month | 上次联系月份 | 'jan', 'feb', 'mar', ..., 'nov', 'dec' |
| day | 上次联系月日 |数字|
| duration | 上次沟通时间 | 数字 |
| campaign | 本次活动已经沟通的次数 | 数字 |
| pdays | 距离上次沟通经过的天数 | 数字 |
| previous | 在本次活动之前已经沟通的次数 | 数字 |
| poutcome | 之前活动的结果 | 'unknown', 'failure', 'other', 'success' | 




每个样本的label - y表示对于目标客户的营销结果（是否签订了定额存款合同），取值是'yes','no'。

我们假定以上16个feature由两个机构分别掌握，具体如下。

- alice: age, job, marital, education, default, balance, housing, loan
- bob: contact, day, month, duration, campaign, pdays, previous, poutcome, y


在真实业务场景中, alice和bob所掌握的数据可能是没有对齐的，为了模拟这种情况，我们将数据集shuffle之后，再随机各取90%来模拟这个状况。

In [3]:
import numpy as np

df_alice = df.iloc[:, np.r_[0:8, -1]].sample(frac=0.9)

df_alice


,age,job,marital,education,default,balance,housing,loan,uid
34307,30,self-employed,married,tertiary,no,2666,no,no,34308
39488,34,management,married,secondary,no,724,yes,no,39489
15177,29,management,married,tertiary,no,1,yes,no,15178
8390,42,technician,single,unknown,no,344,yes,no,8391
24188,58,technician,married,secondary,no,-942,yes,no,24189
...,...,...,...,...,...,...,...,...,...
30242,29,services,married,primary,no,549,yes,no,30243
29936,31,technician,single,tertiary,no,665,yes,no,29937
13661,32,admin.,single,secondary,yes,-17,no,no,13662
8968,41,management,married,tertiary,no,5,no,no,8969


In [4]:
df_bob = df.iloc[:, 8:].sample(frac=0.9)

df_bob


,contact,day,month,duration,campaign,pdays,previous,poutcome,y,uid
21927,cellular,20,aug,210,2,-1,0,unknown,no,21928
11391,unknown,19,jun,573,1,-1,0,unknown,no,11392
26036,cellular,19,nov,266,2,-1,0,unknown,no,26037
43335,cellular,22,mar,254,2,-1,0,unknown,yes,43336
43222,cellular,5,mar,162,1,179,2,failure,no,43223
...,...,...,...,...,...,...,...,...,...,...
28562,cellular,29,jan,146,2,-1,0,unknown,no,28563
37451,cellular,13,may,777,2,-1,0,unknown,no,37452
10843,unknown,17,jun,491,1,-1,0,unknown,no,10844
17215,cellular,28,jul,68,3,-1,0,unknown,no,17216


我们这里将df_alice和df_bob保存为文件，作为alice和bob两方的原始输入。

至此，我们完成了所有实验准备工作。

In [5]:
import tempfile

_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
df_alice.reset_index(drop=True).to_csv(alice_path, index=False)
df_bob.reset_index(drop=True).to_csv(bob_path, index=False)


## 样本对齐（隐私求交）

显然，第一步我们需要将两边的数据对齐。
隐私求交（[Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection))是一种密码学方法，可以获取两个集合的交集，而不泄露任何其他信息。
在隐语中，SPU设备支持三种隐私求交算法:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：半诚实模型, 基于公钥密码学，原本适用于小数据集，但是隐语优化后已经能支持10亿量级的数据。
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：半诚实模型, 基于布谷鸟哈希（Cuckoo Hashing）以及高效不经意传输扩展（OT Extension），适用于大数据集（比如千万数据集）。
- [BC22PCG](https://eprint.iacr.org/2022/334)：半诚实模型, 基于随机相关函数生成器，适用于大数据集。

由于我们这里的数据集较小，我们这里采用的是ECDH方法。


### 方式一：将隐私求交结果保存至文件

在一些应用场景场景中，alice和bob可能在隐私求交之后将结果直接保存至文件中，之后再进行后续操作。这个时候，请调用**psi_csv**接口。

在以下代码中，我们分别制定了两边需要求交的key以及输入和输出路径。

我们需要指定双方的输入文件和输出文件路径。对于ECDH来说，由于双方的地位是平等的，receiver并没有实际含义，你可以任意指定。我们需要设定正确的protocol。sort设为true之后，join的结果将会被排序。

> 请阅读 psi_csv 的文档。

In [6]:
_, alice_psi_path = tempfile.mkstemp()
_, bob_psi_path = tempfile.mkstemp()

spu.psi_csv(
    key="uid",
    input_path={alice: alice_path, bob: bob_path},
    output_path={alice: alice_psi_path, bob: bob_psi_path},
    receiver="alice",
    protocol="ECDH_PSI_2PC",
    sort=True,
)


(pid=12271) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
(pid=12278) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:58:57.878 [info] [bucket_psi.cc:Init:228] bucket size set to 1048576
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:58:57.879 [info] [bucket_psi.cc:Run:97] Begin sanity check for input file: /tmp/tmp694nk0jt, precheck_switch:true
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:58:57.900 [info] [csv_checker.cc:CsvChecker:121] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1687175937879190715 | LC_ALL=C uniq -d > duplicate-keys.1687175937879190715
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:58:57.917 [info] [bucket_psi.cc:Run:115] End sanity check for input file: /tmp/tmp694nk0jt, size=40690
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:58:57.917 [info] [bucket_psi.cc:RunPsi:419] Run psi protocol=1, self_items_count=40690
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-1

[{'party': 'alice', 'original_count': 40690, 'intersection_count': 36623},
 {'party': 'bob', 'original_count': 40690, 'intersection_count': 36623}]

(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:58:58.939 [info] [ecdh_psi.cc:MaskPeer:119] MaskPeer:root--finished, batch_count=10, peer_item_count=40690
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:58:58.976 [info] [bucket_psi.cc:Run:178] Begin post filtering, indices.size=36623, should_sort=true
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:58:58.985 [info] [utils.cc:MultiKeySort:88] Executing sort scripts: tail -n +2 /tmp/tmp-sort-in-1687175938977402931 | LC_ALL=C sort --buffer-size=3G --parallel=8 --temporary-directory=./ --stable --field-separator=, --key=10,10 >>/tmp/tmp-sort-out-1687175938977402931
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:58:59.012 [info] [utils.cc:MultiKeySort:90] Finished sort scripts: tail -n +2 /tmp/tmp-sort-in-1687175938977402931 | LC_ALL=C sort --buffer-size=3G --parallel=8 --temporary-directory=./ --stable --field-separator=, --key=10,10 >>/tmp/tmp-sort-out-1687175938977402931, r

### 方式二：将求交结果保存至VDataFrame

VDataFrame是隐语中保存垂直切分数据的数据结构，在接下来的任务中，我们将会不断使用VDataFrame的数据结构。

由于在本次实验中，经过隐私求交之后，我们还有后续操作，所以我们在这里使用 **data.vertical.read_csv** 来将原始数据隐私求交之后的结果直接转化为VDataFrame。

> 请阅读data.vertical.read_csv的文档。很多参数和psi_csv是一致的，这里不再赘述。

In [7]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv(
    {alice: alice_path, bob: bob_path},
    spu=spu,
    keys="uid",
    drop_keys="uid",
    psi_protocl="ECDH_PSI_2PC",
)
vdf.columns


(pid=4578) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:59:01.035 [info] [bucket_psi.cc:Init:228] bucket size set to 1048576
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:59:01.035 [info] [bucket_psi.cc:Run:97] Begin sanity check for input file: /tmp/tmp694nk0jt, precheck_switch:true
(SPURuntime(device_id=None, party=alice) pid=12271) 2023-06-19 19:59:01.058 [info] [csv_checker.cc:CsvChecker:121] Executing duplicated scripts: LC_ALL=C sort --buffer-size=1G --temporary-directory=/tmp --stable selected-keys.1687175941035742943 | LC_ALL=C uniq -d > duplicate-keys.1687175941035742943
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:59:01.035 [info] [bucket_psi.cc:Init:228] bucket size set to 1048576
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:59:01.035 [info] [bucket_psi.cc:Run:97] Begin sanity check for input file: /tmp/tmpuk_ondd_, precheck_switch:true
(SPURuntime(device_id=None, party=bob) pid=12278) 2023-06-19 19:59:01.05

(pid=4620) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

### 更多

我们在这里展示的是两方单键的隐私求交，隐语也支持三方和多键的隐私求交技术，想要了解更多信息，你可以：

- 阅读这篇[文档](https://www.secretflow.org.cn/docs/spu/en/development/psi.html)了解隐语SPU的隐私求交能力。
- 阅读该[教程](./PSI_On_SPU.ipynb)了解使用的例子。

## 特征预处理

一般情况下，我们都需要对用于建模的数据进行预处理，合理的预处理对模型训练效果非常关键。

在开始特征预处理之前，我们先使用 **stats.table_statistics.table_statistics** 来查看一下特征总体情况，我们会在后面专门讨论全表统计模块。

In [8]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats

(_run pid=4620) /home/t4/fengjun.feng/repos/secretflow/secretflow/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
(_run pid=4620)   return fn(*args, **kwargs)
(_run pid=4620) /home/t4/fengjun.feng/repos/secretflow/secretflow/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
(_run pid=4620)   return fn(*args, **kwargs)
(_run pid=4578) /home/t4/fengjun.feng/repos/secretflow/secretflow/secretflow/device/device/pyu.py:154: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns 

,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,var(variance),std(standard deviation),...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,int64,36623,36623,0,0.0,18.0,95.0,40.973842,1.132227e+02,10.640615,...,1.792075e+03,8.352986e+04,4.136732e+06,1.132196e+02,8.235717e+02,4.271965e+04,1500585.0,6.563117e+07,3.059114e+09,1.514995e+11
job,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
default,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
balance,int64,36623,36623,0,0.0,-8019.0,102127.0,1364.172569,9.389804e+06,3064.278642,...,1.125051e+07,2.846731e+11,4.381241e+15,9.389547e+06,2.437076e+11,1.301696e+16,49960092.0,4.120276e+11,1.042558e+16,-5.566494e+18
housing,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
contact,object,36623,36623,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
day,int64,36623,36623,0,0.0,1.0,31.0,15.802528,6.931243e+01,8.325409,...,3.190304e+02,7.285526e+03,1.789105e+05,6.931054e+01,5.347452e+01,9.321046e+03,578736.0,1.168385e+07,2.668178e+08,6.552241e+09


In [9]:
pd.reset_option('display.max_rows')

在接下来，我们将会展示隐语以下特征预处理能力：

- 值替换
- 缺失值填充
- WOE分组/分箱转换
- one-hot编码
- 标准化

### 值替换

我们先对以下特征做值替换：

| feature | 描述 | 取值和值替换规则 |
| :-----| :---- | :---- |
| education | 教育状况 | 'tertiary' -> 3, 'secondary' -> 2, 'unknown' -> 0, 'primary' -> 1 |
| default | 是否有不良信用记录 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| housing | 是否有房贷 |  'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| loan | 是否有个人贷款 | 'no' -> 0,'yes' -> 1,'unknown' -> NaN |
| month | 上次联系月份 | 'jan' -> 1, 'feb' -> 2, 'mar' -> 3, ..., 'nov' -> 11, 'dec' ->12 |
| y | label | 'yes' -> 1,'no' -> 0 |


替换完之后，我们使用 **sf.reveal** 来查看效果，请注意在生产中，**sf.reveal** 将会直接泄露数据，需要严格限制和进行审计。

> 在生产中，请严格限制**sf.reveal**的使用。

In [10]:
vdf['education'] = vdf['education'].replace(
    {'tertiary': 3, 'secondary': 2, 'primary': 1, 'unknown': np.NaN}
)

vdf['default'] = vdf['default'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['housing'] = vdf['housing'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['loan'] = vdf['loan'].replace({'no': 0, 'yes': 1, 'unknown': np.NaN})

vdf['month'] = vdf['month'].replace(
    {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
)

vdf['y'] = vdf['y'].replace(
    {
        'no': 0,
        'yes': 1,
    }
)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       33     technician   single        2.0        0       56        0     0
3       42     technician  married        2.0        0     8036        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36618   46     technician  married        2.0        0      399        1     0
36619   36  self-employed   single        3.0        0     4844        0     0
36620   49     technician  married        2.0        0      378        0     0
36621   49      housemaid  married        1.0        0     3376        0     0
36622   40    blue-collar  married        1.0        0       48        0     0

[36623 rows x 8 columns]
       contact  day  month

#### 安全性讨论

值替换操作由数据所有者的PYU Device执行，不会泄露数据。

### 缺失值填充

接下来我们对缺失值进行填充。我们在这里均填充了众数，其他可选的策略还包括平均数、中位数等。

其他可能的处理方法包括删除缺省的行, 或者可以使用数据完整的行作为训练集，以此来预测缺失值。

替换完之后，我们使用 **sf.reveal** 来查看效果。

In [11]:
vdf["education"] = vdf["education"].fillna(vdf["education"].mode())
vdf["default"] = vdf["default"].fillna(vdf["default"].mode())
vdf["housing"] = vdf["housing"].fillna(vdf["housing"].mode())
vdf["loan"] = vdf["loan"].fillna(vdf["loan"].mode())

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       33     technician   single        2.0        0       56        0     0
3       42     technician  married        2.0        0     8036        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36618   46     technician  married        2.0        0      399        1     0
36619   36  self-employed   single        3.0        0     4844        0     0
36620   49     technician  married        2.0        0      378        0     0
36621   49      housemaid  married        1.0        0     3376        0     0
36622   40    blue-collar  married        1.0        0       48        0     0

[36623 rows x 8 columns]
       contact  day  month

#### 安全性讨论

所填充的缺失值由属于数据所有者的PYU Device执行，并在接下来的缺失值操作中由数据所有者的PYU Device使用，不会泄露数据。

### woe分箱

woe分箱用于将连续值替换为离散值。

将连续型特征离散化的一个好处是可以有效地克服数据中隐藏的缺陷： 使模型结果更加稳定。例如，数据中的极端值是影响模型效果的一个重要因素。极端值导致模型参数过高或过低，或导致模型被虚假现象"迷惑"，把原来不存在的关系作为重要模式来学习。而离散化可以有效地减弱极端值和异常值的影响。

变量duration的75%分位数远小于最大值，而且该变量的标准差相对也比较大。因此需要对变量duration进行离散化。

In [12]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_woe_substitution import VertWOESubstitution

binning = VertWoeBinning(spu)
woe_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: [], bob: ["duration"]},
    label_name="y",
)

woe_sub = VertWOESubstitution()
vdf = woe_sub.substitution(vdf, woe_rules)

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party bob.
(pid=17408) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
(pid=17366) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_substitution.VertWOESubstitutionPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_substitution.VertWOESubstitutionPyuWorker'> with party bob.
(_run pid=4620) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=4620) INFO:jax._src.

       age            job  marital  education  default  balance  housing  loan
0       58     management  married        3.0        0     2143        1     0
1       43     technician   single        2.0        0      593        1     0
2       33     technician   single        2.0        0       56        0     0
3       42     technician  married        2.0        0     8036        0     0
4       38         admin.  married        1.0        0     1487        0     0
...    ...            ...      ...        ...      ...      ...      ...   ...
36618   46     technician  married        2.0        0      399        1     0
36619   36  self-employed   single        3.0        0     4844        0     0
36620   49     technician  married        2.0        0      378        0     0
36621   49      housemaid  married        1.0        0     3376        0     0
36622   40    blue-collar  married        1.0        0       48        0     0

[36623 rows x 8 columns]
       contact  day  month

#### 安全性讨论

woe分桶需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### One Hot编码

one-hot编码适用于将类型编码转化为数值编码。 对于job、marital等特征我们需要one-hot编码。

In [13]:
from secretflow.preprocessing.encoder import OneHotEncoder

encoder = OneHotEncoder()
# for vif and correlation only
vdf_hat = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

tranformed_df = encoder.fit_transform(vdf['job'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['marital'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['contact'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['month'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['day'])
vdf[tranformed_df.columns] = tranformed_df

tranformed_df = encoder.fit_transform(vdf['poutcome'])
vdf[tranformed_df.columns] = tranformed_df

vdf = vdf.drop(columns=["job", "marital", "contact", "month", "day", "poutcome"])

print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


       age  education  default  balance  housing  loan  job_admin.  \
0       58        3.0        0     2143        1     0         0.0   
1       43        2.0        0      593        1     0         0.0   
2       33        2.0        0       56        0     0         0.0   
3       42        2.0        0     8036        0     0         0.0   
4       38        1.0        0     1487        0     0         1.0   
...    ...        ...      ...      ...      ...   ...         ...   
36618   46        2.0        0      399        1     0         0.0   
36619   36        3.0        0     4844        0     0         0.0   
36620   49        2.0        0      378        0     0         0.0   
36621   49        1.0        0     3376        0     0         0.0   
36622   40        1.0        0       48        0     0         0.0   

       job_blue-collar  job_entrepreneur  job_housemaid  ...  job_retired  \
0                  0.0               0.0            0.0  ...          0.0   
1    

#### 安全性讨论

one-hot编码操作由数据所有者的PYU Device执行，不会泄露数据。

### 标准化 
特征之间数值差距太大会使得模型收敛困难，我们一般先对数值进行标准化。

In [14]:
from secretflow.preprocessing import StandardScaler

X = vdf.drop(columns=['y'])
y = vdf['y']
scaler = StandardScaler()
X = scaler.fit_transform(X)
vdf[X.columns] = X
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))


(_run pid=4609) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4609)   warnings.warn(
(_run pid=4620) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4620)   warnings.warn(
(pid=4610) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


            age  education   default   balance   housing      loan  \
0      1.600132   1.317305 -0.135366  0.254167  0.896115 -0.438311   
1      0.190420  -0.216431 -0.135366 -0.251669  0.896115 -0.438311   
2     -0.749388  -0.216431 -0.135366 -0.426916 -1.115928 -0.438311   
3      0.096439  -0.216431 -0.135366  2.177321 -1.115928 -0.438311   
4     -0.279484  -1.750166 -0.135366  0.040084 -1.115928 -0.438311   
...         ...        ...       ...       ...       ...       ...   
36618  0.472362  -0.216431 -0.135366 -0.314980  0.896115 -0.438311   
36619 -0.467446   1.317305 -0.135366  1.135626 -1.115928 -0.438311   
36620  0.754305  -0.216431 -0.135366 -0.321833 -1.115928 -0.438311   
36621  0.754305  -1.750166 -0.135366  0.656551 -1.115928 -0.438311   
36622 -0.091522  -1.750166 -0.135366 -0.429527 -1.115928 -0.438311   

       job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  ...  \
0       -0.359042         -0.52347         -0.184134      -0.167632  ...   
1      

#### 安全性讨论

标准化操作由数据所有者的PYU Device执行，不会泄露数据。

### 更多

隐语还支持其他更多的特征预处理能力，请参考这篇[文档](./data_preprocessing_with_data_frame.ipynb).

至此，我们已经完成了所有特征预处理工作。

> 本文主要目的是为了展示隐语的预处理能力，本文对于数据预处理方法的使用可能是有争议的，敬请谅解。

## 数据分析

在建模之前，我们有必要分析一下我们所使用的数据，以便确认是否需要重复特征预处理的过程。

下面我们将会展示隐语以下数据分析能力:

- 全表统计
- 相关系数矩阵
- VIF指标计算


### 全表统计

我们提供了类似于 **pd.DataFrame.describe** 来展示所有特征的基本统计信息。

> 在特征预处理的过程中，你可以不断调用全表统计来关注预处理效果。

In [15]:
from secretflow.stats.table_statistics import table_statistics

pd.set_option('display.max_rows', None)
data_stats = table_statistics(vdf)
data_stats


,datatype,total_count,count(non-NA count),count_na(NA count),na_ratio,min,max,mean,var(variance),std(standard deviation),...,moment_2,moment_3,moment_4,central_moment_2,central_moment_3,central_moment_4,sum,sum_2,sum_3,sum_4
age,float64,36623,36623,0,0.0,-2.159100,5.077423,1.490039e-16,1.000027,1.000014,...,1.000000,0.683627,3.332610,1.000000,0.683627,3.332610,5.456968e-12,36623.0,2.503646e+04,1.220502e+05
education,float64,36623,36623,0,0.0,-1.750166,1.317305,-2.235058e-16,1.000027,1.000014,...,1.000000,-0.150311,2.309413,1.000000,-0.150311,2.309413,-8.185452e-12,36623.0,-5.504831e+03,8.457763e+04
default,float64,36623,36623,0,0.0,-0.135366,7.387394,-2.483398e-17,1.000027,1.000014,...,1.000000,7.252029,53.591920,1.000000,7.252029,53.591920,-9.094947e-13,36623.0,2.655910e+05,1.962697e+06
balance,float64,36623,36623,0,0.0,-3.062156,32.883499,3.880309e-17,1.000027,1.000014,...,1.000000,8.470364,147.645486,1.000000,8.470364,147.645486,1.421085e-12,36623.0,3.102101e+05,5.407221e+06
housing,float64,36623,36623,0,0.0,-1.115928,0.896115,2.110888e-16,1.000027,1.000014,...,1.000000,-0.219812,1.048317,1.000000,-0.219812,1.048317,7.730705e-12,36623.0,-8.050184e+03,3.839253e+04
loan,float64,36623,36623,0,0.0,-0.438311,2.281487,7.062162e-17,1.000027,1.000014,...,1.000000,1.843177,4.397301,1.000000,1.843177,4.397301,2.586376e-12,36623.0,6.750267e+04,1.610423e+05
job_admin.,float64,36623,36623,0,0.0,-0.359042,2.785192,-2.561004e-17,1.000027,1.000014,...,1.000000,2.426150,6.886204,1.000000,2.426150,6.886204,-9.379164e-13,36623.0,8.885289e+04,2.521935e+05
job_blue-collar,float64,36623,36623,0,0.0,-0.523470,1.910330,-4.035521e-17,1.000027,1.000014,...,1.000000,1.386860,2.923380,1.000000,1.386860,2.923380,-1.477929e-12,36623.0,5.079096e+04,1.070629e+05
job_entrepreneur,float64,36623,36623,0,0.0,-0.184134,5.430815,1.940154e-18,1.000027,1.000014,...,1.000000,5.246681,28.527661,1.000000,5.246681,28.527661,7.105427e-14,36623.0,1.921492e+05,1.044769e+06
job_housemaid,float64,36623,36623,0,0.0,-0.167632,5.965435,-5.432432e-18,1.000027,1.000014,...,1.000000,5.797803,34.614514,1.000000,5.797803,34.614514,-1.989520e-13,36623.0,2.123329e+05,1.267687e+06


In [16]:
pd.reset_option('display.max_rows')


#### 安全性讨论

请注意，全表统计会暴露数据整体统计结果，其背后实际上蕴含了**sf.reveal**,请谨慎使用。

### 相关系数矩阵

我们接下来计算特征和特征之间，特征和标签之间的相关系数矩阵。

> 计算相关系数矩阵时，one-hot编码各列无需参与计算。

In [17]:
from secretflow.stats.ss_pearsonr_v import PearsonR

pearson_r_calculator = PearsonR(spu)
corr_matrix = pearson_r_calculator.pearsonr(vdf_hat)

import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
corr_matrix


(_run pid=4620) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4620)   warnings.warn(
(_run pid=4610) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4610)   warnings.warn(
(_run pid=4610) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=4610) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=4610) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=4610) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_clien

(_run pid=4620) [2023-06-19 19:59:23.463] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(_run pid=4610) [2023-06-19 19:59:23.478] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


array([[1.000, -0.162, -0.018, 0.096, -0.186, -0.017, -0.011, 0.006,
        -0.027, -0.000, 0.028],
       [-0.162, 1.000, -0.013, 0.068, -0.075, -0.022, -0.004, 0.003,
        0.012, 0.030, 0.066],
       [-0.018, -0.013, 1.000, -0.066, -0.006, 0.078, -0.007, 0.012,
        -0.028, -0.017, -0.020],
       [0.096, 0.068, -0.066, 1.000, -0.068, -0.084, 0.015, -0.012,
        0.004, 0.016, 0.050],
       [-0.186, -0.075, -0.006, -0.068, 1.000, 0.040, 0.002, -0.028,
        0.123, 0.034, -0.137],
       [-0.017, -0.022, 0.078, -0.084, 0.040, 1.000, -0.012, 0.013,
        -0.023, -0.009, -0.068],
       [-0.011, -0.004, -0.007, 0.015, 0.002, -0.012, 1.000, -0.173,
        0.016, 0.011, 0.325],
       [0.006, 0.003, 0.012, -0.012, -0.028, 0.013, -0.173, 1.000,
        -0.089, -0.032, -0.072],
       [-0.027, 0.012, -0.028, 0.004, 0.123, -0.023, 0.016, -0.089,
        1.000, 0.439, 0.106],
       [-0.000, 0.030, -0.017, 0.016, 0.034, -0.009, 0.011, -0.032,
        0.439, 1.000, 0.092],
    

#### 安全性讨论

相关系数矩阵的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

### VIF指标计算

隐语还支持VIF的计算来进行多重共线性检验。

> 计算VIF指标时，one-hot编码各列无需参与计算。

In [18]:
from secretflow.stats.ss_vif_v import VIF

vif_calculator = VIF(spu)
vif_results = vif_calculator.vif(vdf_hat)
print(vdf_hat.columns)
print(vif_results)


(_run pid=4610) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4610)   warnings.warn(
(_run pid=4620) /home/fengjun.feng/anaconda3/envs/sf/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(_run pid=4620)   warnings.warn(


Index(['age', 'education', 'default', 'balance', 'housing', 'loan', 'duration',
       'campaign', 'pdays', 'previous', 'y'],
      dtype='object')
[1.081 1.051 1.011 1.030 1.091 1.018 1.151 1.040 1.276 1.243 1.173]


#### 安全性讨论

VIF指标的计算需要利用alice和bob两边的数据，因此相关的计算需要使用**SPU device**确保原始数据不被泄露。

## 模型训练

接下来，我们将会分别训练一个逻辑回归模型和一个XGB模型。


### 随机分割

在训练之前，我们需要将数据分割为训练集和验证集。

其中train_x和train_y为训练集的特征和标签。test_x和test_y为训练集的特征和标签。


In [19]:
from secretflow.data.split import train_test_split

random_state = 1234

train_vdf, test_vdf = train_test_split(vdf, train_size=0.8, random_state=random_state)

train_x = train_vdf.drop(columns=['y'])
train_y = train_vdf['y']

test_x = test_vdf.drop(columns=['y'])
test_y = test_vdf['y']


#### 安全性讨论

随机分割时，每一方会共享随机数种子，并由每一方数据的owner分别执行各自的数据分割并且确保最终分割结果仍然是对齐的。

### PSI（人群稳定性分析）

样本稳定指数是衡量样本变化所产生的偏移量的一种重要指标，通常用来衡量样本的稳定程度，比如样本在两个月份之间的变化是否稳定。通常变量的PSI值在0.1以下表示变化不太显著，在0.1到0.25之间表示有比较显著的变化，大于0.25表示变量变化比较剧烈，需要特殊关注。

接下来以`balance`为例子，确认两次抽样的样本分布是否接近。

> 根据业务需求，PSI分析也可以在数据分析或者特征预处理的时候进行。


In [20]:
stats_df = table_statistics(train_x['balance'])

In [21]:
min_val, max_val = stats_df['min'], stats_df['max']

In [22]:
from secretflow.stats import psi_eval
from secretflow.stats.core.utils import equal_range
import jax.numpy as jnp

split_points = equal_range(jnp.array([min_val, max_val]), 3)
balance_psi_score = psi_eval(train_x['balance'], test_x['balance'], split_points)

sf.reveal(balance_psi_score)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.


Array(0.000, dtype=float32)

#### 安全性讨论

PSI分析是一个单方运算，由数据owner的PYU Device执行计算。

### 逻辑回归模型

使用 **ml.linear.ss_sgd.SSRegression** 可以进行密态逻辑回归模型的训练。

请参考相关的API文档。


In [23]:
from secretflow.ml.linear.ss_sgd import SSRegression

lr_model = SSRegression(spu)
lr_model.fit(
    x=train_x,
    y=train_y,
    epochs=3,
    learning_rate=0.1,
    batch_size=1024,
    sig_type='t1',
    reg_type='logistic',
    penalty='l2',
    l2_norm=0.5,
)


(_run pid=4609) [2023-06-19 19:59:26.102] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


INFO:root:epoch 1 times: 1.8808467388153076s
INFO:root:epoch 2 times: 0.7152931690216064s
INFO:root:epoch 3 times: 0.6313662528991699s


你可能会对为何上面的语句很快就执行完毕感到困惑，原因是在隐语中，语句都是lazy evaluation的，在上面的例子中，直到lr_model被真正被使用的时候，**lr_model.fit**才会被执行。

#### 安全性讨论

SSRegression的训练基于SPU Device，双方的原始数据将会被保护。

### XGBoost模型

使用 **ml.boost.ss_xgb_v.Xgb** 可以进行密态XGBoost模型的训练。

请参考相关的API文档。

In [24]:
from secretflow.ml.boost.ss_xgb_v import Xgb

xgb = Xgb(spu)
params = {
    'num_boost_round': 3,
    'max_depth': 5,
    'sketch_eps': 0.25,
    'objective': 'logistic',
    'reg_lambda': 0.2,
    'subsample': 1,
    'colsample_by_tree': 1,
    'base_score': 0.5,
}
xgb_model = xgb.train(params=params, dtrain=train_x, label=train_y)


INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.
INFO:root:fragment_count 1
INFO:root:prepare time 0.17217350006103516s
(pid=28283) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
(pid=28284) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
INFO:root:global_setup time 3.0523507595062256s
INFO:root:build & infeed bucket_map fragments [0, 0]
INFO:root:build & infeed bucket_map time 0.8674089908599854s
INFO:root:init_pred time 0.0363163948059082s
INFO:root:epoch 0 tree_setup time 0.09707164764404297s
INFO:root:fragment[0, 0] gradient sum time 1.1570940017700195s
INFO:root:level 0 time 1.4122493267059326s
INFO:root:fragment[0, 0] gradient sum time 1.88447332

(XgbTreeWorker pid=28283) [2023-06-19 20:00:03.794] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(XgbTreeWorker pid=28284) [2023-06-19 20:00:03.816] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


INFO:root:epoch 0 time 30.738550424575806s
INFO:root:epoch 1 tree_setup time 0.5257089138031006s
INFO:root:fragment[0, 0] gradient sum time 1.7339739799499512s
INFO:root:level 0 time 1.875471830368042s
INFO:root:fragment[0, 0] gradient sum time 1.8286621570587158s
INFO:root:level 1 time 2.0614569187164307s
INFO:root:fragment[0, 0] gradient sum time 3.5874507427215576s
INFO:root:level 2 time 3.8770413398742676s
INFO:root:fragment[0, 0] gradient sum time 7.182074069976807s
INFO:root:level 3 time 7.561537027359009s
INFO:root:fragment[0, 0] gradient sum time 14.358884334564209s
INFO:root:level 4 time 14.837937831878662s
INFO:root:epoch 1 time 30.98349380493164s
INFO:root:epoch 2 tree_setup time 0.525902271270752s
INFO:root:fragment[0, 0] gradient sum time 1.7702536582946777s
INFO:root:level 0 time 1.983457088470459s
INFO:root:fragment[0, 0] gradient sum time 1.7969560623168945s
INFO:root:level 1 time 1.9531784057617188s
INFO:root:fragment[0, 0] gradient sum time 3.668266534805298s
INFO:roo

Xgb.train将会直接执行，请耐心等待。

#### 安全性讨论

Xgb的训练基于SPU Device，双方的原始数据将会被保护。

## 模型预测

接下来，我们将会分别利用刚刚训练好的模型来预测测试集。

### 逻辑回归模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [25]:
lr_y_hat = lr_model.predict(x=test_x, batch_size=1024, to_pyu=bob)

#### 安全性讨论

逻辑回归的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

### XGBoost模型

由于在我们的场景下，数据集标签的持有者是bob，因此我们在这里将预测结果**reveal**给bob.

In [26]:
xgb_y_hat = xgb_model.predict(dtrain=test_x, to_pyu=bob)


INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.
(pid=51208) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
(pid=51207) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


(XgbTreeWorker pid=51207) [2023-06-19 20:01:09.580] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63
(XgbTreeWorker pid=51208) [2023-06-19 20:01:09.595] [info] [thread_pool.cc:30] Create a fixed thread pool with size 63


#### 安全性讨论

XGBoost模型的预测基于SPU Device，双方的原始数据将会被保护。

当设置**to_pyu**，预测结果将会被reveal给该方，否则将仍然保持秘密分享的状态。

## 模型评估

接下来，我们将利用测试数据集对模型效果进行评估，包括：

- 二分类评估
- PVA
- P-Value
- 评分卡转换

### 二分类评估

隐语中对二分类的评估有集成的支持。

`BiClassificationEval` 将计算 `AUC`, `KS`, `F1 Score`, `Lift`, `K-S`, `Gain`, `Precision`, `Recall` 等统计数值， 并提供（基于prediction score的）等频和等距分箱的统计报告和总报告。

不同分桶中评估模型的预测的`threshold`不同。总报告中依赖`threshold`的统计取的是各个分桶的最佳值。

详情可以参考API文档。

In [27]:
from secretflow.stats.biclassification_eval import BiClassificationEval

biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=lr_y_hat, bucket_size=20
)
lr_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [28]:
print(f'positive_samples: {lr_report.summary_report.positive_samples}')
print(f'negative_samples: {lr_report.summary_report.negative_samples}')
print(f'total_samples: {lr_report.summary_report.total_samples}')
print(f'auc: {lr_report.summary_report.auc}')
print(f'ks: {lr_report.summary_report.ks}')
print(f'f1_score: {lr_report.summary_report.f1_score}')


positive_samples: 884.0
negative_samples: 6441.0
total_samples: 7325.0
auc: 0.8973933458328247
ks: 0.6497097015380859
f1_score: 0.5536627173423767


In [29]:
biclassification_evaluator = BiClassificationEval(
    y_true=test_y, y_score=xgb_y_hat, bucket_size=20
)
xgb_report = sf.reveal(biclassification_evaluator.get_all_reports())


In [30]:
print(f'positive_samples: {xgb_report.summary_report.positive_samples}')
print(f'negative_samples: {xgb_report.summary_report.negative_samples}')
print(f'total_samples: {xgb_report.summary_report.total_samples}')
print(f'auc: {xgb_report.summary_report.auc}')
print(f'ks: {xgb_report.summary_report.ks}')
print(f'f1_score: {xgb_report.summary_report.f1_score}')


positive_samples: 884.0
negative_samples: 6441.0
total_samples: 7325.0
auc: 0.8754899501800537
ks: 0.5985674858093262
f1_score: 0.5030487775802612


### 预测偏差

结果由`abs(mean(Acutal) - mean(Prediction))`计算获得, 值越小越好。

In [31]:
from secretflow.stats import prediction_bias_eval

prediction_bias = prediction_bias_eval(test_y, lr_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width')

sf.reveal(prediction_bias)


PredictionBiasReport(buckets=[BucketPredictionBiasReport(left_endpoint=0.0, left_closed=True, right_endpoint=0.25, right_closed=False, isna=False, avg_prediction=0.0, avg_label=0.13951896131038666, bias=0.13951896131038666, absolute=True), BucketPredictionBiasReport(left_endpoint=0.25, left_closed=True, right_endpoint=0.5, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.5, left_closed=True, right_endpoint=0.75, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.75, left_closed=True, right_endpoint=1.0, right_closed=True, isna=False, avg_prediction=1.0, avg_label=0.36417245864868164, bias=0.6358275413513184, absolute=True)])

In [32]:
xgb_pva_score = prediction_bias_eval(test_y, xgb_y_hat, bucket_num=4, absolute=True, bucket_method='equal_width')

sf.reveal(xgb_pva_score)


PredictionBiasReport(buckets=[BucketPredictionBiasReport(left_endpoint=0.0, left_closed=True, right_endpoint=0.25, right_closed=False, isna=False, avg_prediction=0.0, avg_label=0.1599043607711792, bias=0.1599043607711792, absolute=True), BucketPredictionBiasReport(left_endpoint=0.25, left_closed=True, right_endpoint=0.5, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.5, left_closed=True, right_endpoint=0.75, right_closed=False, isna=True, avg_prediction=0, avg_label=0, bias=0, absolute=True), BucketPredictionBiasReport(left_endpoint=0.75, left_closed=True, right_endpoint=1.0, right_closed=True, isna=False, avg_prediction=1.0, avg_label=0.370947003364563, bias=0.629052996635437, absolute=True)])

### P-Value
双方可通过p-value的值来判断参数是否显著，即该自变量是否可以有效预测因变量的变异, 从而判定对应的解释变量是否应包括在模型中。


In [33]:
from secretflow.stats import SSPValue

model = lr_model.save_model()
sspv = SSPValue(spu)
pvalues = sspv.pvalues(test_x, test_y, model)

pvalues

array([0.729, 0.477, 0.890, 0.463, 0.022, 0.144, 0.997, 0.991, 0.987,
       0.984, 0.999, 0.959, 0.988, 0.992, 0.939, 0.995, 0.982, 0.993,
       0.996, 0.990, 0.990, 0.000, 0.686, 0.796, 0.779, 0.981, 0.990,
       0.976, 0.955, 0.992, 0.790, 0.969, 0.978, 0.974, 0.976, 0.976,
       0.836, 0.820, 0.985, 0.830, 0.972, 0.984, 0.995, 0.990, 0.996,
       0.984, 0.973, 0.994, 0.996, 0.953, 0.985, 0.977, 0.964, 0.994,
       0.984, 0.984, 0.964, 0.999, 0.966, 0.978, 0.992, 0.982, 0.976,
       0.994, 0.976, 0.993, 0.969, 0.995, 0.983, 0.963, 0.987, 0.976,
       0.992, 0.824, 0.979, 0.000])

### 评分卡转换

> 严格来说，评分卡转化是对预测结果的后续处理，并不属于模型评估。


我们将 `y = 1` 的概率设为`p`， `odds = p / (1 - p)`, 评分卡设定的分值刻度可以通过将分值表示为比率对数的线性表达式来定义，即可表示为下式：

`Score = A - B log(odds)`， A 和 B 是可以设定的常数。隐语中提供了评分卡转换功能，详情可以参考API文档。

In [34]:
from secretflow.stats import BiClassificationEval, ScoreCard

sc = ScoreCard(20, 600, 20)
score = sc.transform(xgb_y_hat)

sf.reveal(score.partitions[bob])


array([[456.557],
       [453.931],
       [487.774],
       ...,
       [456.557],
       [487.774],
       [453.931]])

### 安全性讨论

以上所有模型评估的方法均为单方运算，由label拥有者的PYU Device进行运算。

## 实验结束

最后，我们需要清理临时文件，并关闭隐语cluster。

In [35]:
import os

try:
    os.remove(alice_path)
    os.remove(alice_psi_path)
    os.remove(bob_path)
    os.remove(bob_psi_path)
except OSError:
    pass

sf.shutdown()

恭喜！你已经完成了隐语金融风控全链路的全部实验内容。

如果你对本实验有任何建议和问题，请在[Github Issues](https://github.com/secretflow/secretflow/issues)上联系我们。